In [ ]:
#default_exp rest

In [ ]:
#export
import requests
import json
from nifi_api.environment import NifiEndpoint, Credentials
from nifi_api.tools import custom_response

# REST
> The NIFI REST API calls

## Processors
> NIFI REST API for Connections

In [ ]:
#export


class Processor:

    nifi_api = NifiEndpoint.processors

    def __init__(self, processor_id: str) -> None:
        self.processor_id = processor_id

    def get_state(self) -> None:
        self.state, _ = self.get(self.processor_id)
        self.client_id = self.state['revision']['clientId']

    def get(self, processor_id) -> tuple:
        url = self.nifi_api + processor_id
        res = requests.get(url, auth=Credentials.credentials)
        return custom_response(res)

    def update_run_status(self, state) -> tuple:
        """
Change the status of the processor.

 Attributes
----------
state : str
     The possible values are: "RUNNING", "STOPPED", "DISABLED".

 Returns
----------
 custom_response : tuple
   The response as defined in tools module
"""

        self.get_state()

        data = {
            "revision": {
                'clientId': self.client_id,
                'version': 1
            },
            "state": state,
            "disconnectedNodeAcknowledged": True
        }

        url = self.nifi_api + f"{self.processor_id}/run-status"

        res = requests.put(url,
                           data=json.dumps(data),
                           auth=Credentials.credentials,
                           headers={'content-type': 'application/json'})
        return custom_response(res)

In [ ]:
#test

from nifi_api.environment import NifiIds
p = Processor(NifiIds.sourceToRaw.in_processor)

In [ ]:
p.processor_id

'1ac4364b-5f00-16b3-aee1-e15d8ad527ff'

In [ ]:
p.get_state()

In [ ]:
p.client_id

'cac6ae96-0177-1000-4de6-1ff55bf468ae'

In [ ]:
_,r= p.update_run_status(state="STOPPED")

In [ ]:
r

<Response [200]>

## Connections
> NIFI REST API for Connections

In [ ]:
#export
class Connection:

    nifi_api = NifiEndpoint.connections

    @classmethod
    def list_queues(cls, processor_id):
        url = cls.nifi_api + processor_id
        res = requests.get(url, auth=Credentials.credentials)
        return custom_response(res)

## FlowFile Queues
> NIFI REST API for Flowfile Queues

In [ ]:
#export
class Flowfile:

    @classmethod
    def listing_request(cls, connection_id):
        url = NifiEndpoint.flowfile_queues + f"{connection_id}/listing-requests"
        res = requests.post(url, auth=Credentials.credentials)
        return custom_response(res)

    @classmethod
    def list_queues(cls, connection_id):
        listing_request, _ = cls.listing_request(connection_id)
        listing_request_id = listing_request["listingRequest"]['id']
        endpoint = f"{connection_id}/listing-requests/{listing_request_id}"
        url = NifiEndpoint.flowfile_queues + endpoint
        res = requests.get(url, auth=Credentials.credentials)
        return custom_response(res)

In [ ]:
class Vars:
    time_between_checks = environ['']